In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import dask.dataframe as dd
from catboost import CatBoostRegressor, Pool
import os
import seaborn as sns
import matplotlib.pylab as plt
from pandas.plotting import autocorrelation_plot

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
pd.options.display.max_columns = 999

## Data load

In [ ]:
calendar_df = pd.read_csv("data/calendar.csv", low_memory=False)
sales_df = pd.read_csv("data/sales_train_validation.csv", low_memory=False)
prices_df = pd.read_csv("data/sell_prices.csv", low_memory=False)

In [ ]:
sample_submission = pd.read_csv("data/sample_submission.csv", low_memory=False)

In [ ]:
sales_denorm_df = (
    sales_df
#     .sample(frac=0.2)
    .melt(
        id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
        var_name="d",
        value_name="sales"
    )
    .merge(calendar_df, on=["d"])
    .merge(prices_df, on=["store_id", "item_id", "wm_yr_wk"])
    .assign(total_sales=lambda x: x["sales"] * x["sell_price"])
    .assign(day=lambda x: x["d"].str.slice(start=2).astype(int))
    .assign(date=lambda x: dd.to_datetime(x["date"]))
    .assign(dayofyear=lambda x: x["date"].dt.dayofyear)
    .assign(dayofmonth=lambda x: x["date"].dt.day)
    .drop(columns=["d"])
)

In [ ]:
sales_denorm_df.head()

## Data exploration
- are there missing rows in the time series from when the product was not sold?
- are there missing or zero-valued prices?
- is the data zero-inflated?
- how to incorporate events?
- how to incorporate time series trends?

In [ ]:
sales_denorm_df.describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])

In [ ]:
sales_denorm_df.select_dtypes(exclude=["number"]).describe()

## Model build

In [ ]:
train_df, val_df = train_test_split(sales_denorm_df, train_size=0.5)
val_df, test_df = train_test_split(val_df, train_size=0.5)

In [ ]:
complete_pool = Pool(
    data=sales_denorm_df[["dayofyear", "year", "month", "dayofmonth", "wday", "dept_id", "item_id", "store_id", "sell_price"]], 
    label=sales_denorm_df["sales"],
    weight=sales_denorm_df["sell_price"],
    cat_features=["dept_id", "store_id", "item_id"],
)

In [ ]:
train_pool = Pool(
    data=train_df[["dayofyear", "year", "month", "dayofmonth", "wday", "dept_id", "item_id", "store_id", "sell_price"]], 
    label=train_df["sales"],
    weight=train_df["sell_price"],
    cat_features=["dept_id", "store_id", "item_id"],
)

In [ ]:
eval_pool = Pool(
    data=val_df[["dayofyear", "year", "month", "dayofmonth", "wday", "dept_id", "item_id", "store_id", "sell_price"]], 
    label=val_df["sales"],
    weight=val_df["sell_price"],
    cat_features=["dept_id", "store_id", "item_id"],
)

In [ ]:
test_pool = Pool(
    data=test_df[["dayofyear", "year", "month", "dayofmonth", "wday", "dept_id", "item_id", "store_id", "sell_price"]], 
    label=test_df["sales"],
    weight=test_df["sell_price"],
    cat_features=["dept_id", "store_id", "item_id"],
)

In [ ]:
model = CatBoostRegressor(iterations=1000)

In [ ]:
model.fit(
    train_pool,
    eval_set=eval_pool,
    use_best_model=True,
    plot=True,
    early_stopping_rounds=10,
)

In [ ]:
current_time = datetime.now().strftime('%y%m%d%H%M%S')
model.save_model("data/model.cbm", pool=train_pool)
model.save_model(f"data/model_{current_time}.cbm", pool=train_pool)

In [ ]:
print(
    f"Train score: {model.score(train_pool)}, "
    f"Eval score: {model.score(eval_pool)}, "
    f"Test score: {model.score(test_pool)}"
)

## Visualize predictions

In [ ]:
sales_denorm_df["prediction"] = model.predict(complete_pool)

sales_denorm_df["total_sales_prediction"] = (
    sales_denorm_df["prediction"] * sales_denorm_df["sell_price"]
)

sales_denorm_df["error"] = (
    sales_denorm_df["total_sales_prediction"] - sales_denorm_df["total_sales"]
)

sales_denorm_df["abs_error"] = abs(
    sales_denorm_df["total_sales_prediction"] - sales_denorm_df["total_sales"]
)

### Actuals

In [ ]:
sns.relplot(
    x="dayofyear",
    y="total_sales",
    hue="year",
    data=sales_denorm_df.groupby(["year", "dayofyear"], as_index=False)["total_sales"].sum(),
    kind="line",
    height=5,
    aspect=2,
)

### Prediction

In [ ]:
sns.relplot(
    x="dayofyear",
    y="total_sales_prediction",
    hue="year",
    data=sales_denorm_df.groupby(["year", "dayofyear"], as_index=False)["total_sales_prediction"].sum(),
    kind="line",
    height=5,
    aspect=2,
)

### Error

#### Bias

In [ ]:
plt.figure(figsize=(12, 6))
sns.distplot(np.minimum(25, np.maximum(-25, sales_denorm_df["error"])))

In [ ]:
sales_denorm_df["error"].describe()

#### Error autocorrelation

In [ ]:
def autocorr_plot(df):
    plt.figure(figsize=(12, 6))
    ax = plt.gca(ylim=(-0.25, 0.25), xlim=(0, 25))
    autocorrelation_plot(df.groupby(["date"])["error"].mean().sort_index(), ax=ax)
    plt.title(df.cat_id.unique())
    plt.show()

In [ ]:
sales_denorm_df.groupby(["cat_id"]).apply(lambda x: autocorr_plot(x));

In [ ]:
plt.figure(figsize=(12, 6))
autocorrelation_plot(sales_denorm_df.groupby(["date"])["error"].mean().sort_index())
plt.ylim((-0.25,0.25))
plt.xlim((0,25))
plt.show();

#### Error over time

In [ ]:
sns.relplot(
    x="dayofyear",
    y="error",
    hue="year",
    data=sales_denorm_df.groupby(["year", "dayofyear"], as_index=False)["error"].mean(),
    kind="line",
    height=5,
    aspect=2,
)

In [ ]:
sns.relplot(
    x="dayofyear",
    y="error",
    hue="year",
    row="cat_id",
    data=sales_denorm_df.groupby(["year", "dayofyear", "cat_id"], as_index=False)["error"].mean(),
    kind="line",
    height=5,
    aspect=2,
)

#### Absolute error

In [ ]:
sns.relplot(
    x="dayofyear",
    y="abs_error",
    hue="year",
    data=sales_denorm_df.groupby(["year", "dayofyear"], as_index=False)["abs_error"].mean(),
    kind="line",
    height=5,
    aspect=2,
)

## Forecast

### Construct input df

In [ ]:
max_training_date = sales_denorm_df.date.max()

In [ ]:
items_df = sales_df[["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]].drop_duplicates()

In [ ]:
input_df = (
    items_df.assign(key=0)
    .merge(calendar_df.assign(key=0))
    .merge(prices_df, on=["store_id", "item_id", "wm_yr_wk"])
    .assign(day=lambda x: x["d"].str.slice(start=2).astype(int))
    .assign(date=lambda x: dd.to_datetime(x["date"]))
    .assign(dayofyear=lambda x: x["date"].dt.dayofyear)
    .assign(dayofmonth=lambda x: x["date"].dt.day)
    .drop(columns=["d", "key"])
)

In [ ]:
input_df["is_forecasted"] = input_df["date"].gt(max_training_date)

In [ ]:
input_df["days_ahead"] = (input_df["date"] - max_training_date).dt.days

In [ ]:
input_df["submission_days_ahead"] = np.minimum(input_df["days_ahead"], (input_df["days_ahead"] - 1) % 28 + 1)

In [ ]:
input_df["type"] = (
    np.where(input_df["days_ahead"].ge(29), "evaluation", "validation")
)

In [ ]:
len(input_df.date.unique()), len(input_df.item_id.unique())

### Score inputs

In [ ]:
forecast_pool = Pool(
    data=input_df[["dayofyear", "year", "month", "dayofmonth", "wday", "dept_id", "item_id", "store_id", "sell_price"]], 
    weight=input_df["sell_price"],
    cat_features=["dept_id", "store_id", "item_id"],
)

In [ ]:
input_df["prediction"] = model.predict(forecast_pool)

input_df["total_sales_prediction"] = (
    input_df["prediction"] * input_df["sell_price"]
)

In [ ]:
sns.relplot(
    x="dayofyear",
    y="total_sales_prediction",
    hue="year",
    style="is_forecasted",
    data=input_df.groupby(["year", "dayofyear", "is_forecasted"], as_index=False)["total_sales_prediction"].sum(),
    kind="line",
    height=5,
    aspect=2,
)

In [ ]:
sns.relplot(
    x="dayofyear",
    y="total_sales_prediction",
    hue="year",
    data=sales_denorm_df.groupby(["year", "dayofyear"], as_index=False)["total_sales_prediction"].sum(),
    kind="line",
    height=5,
    aspect=2,
)

### Construct submission dataframe

In [ ]:
submission = (
    input_df
    .query("days_ahead > 0")
    .assign(f=lambda x: "F" + x["submission_days_ahead"].astype(str))
    .assign(id=lambda x: x["item_id"] + "_" + x["store_id"] + "_" + x["type"])
    .pivot(index="id", values="prediction", columns="f")[[f"F{i+1}" for i in range(28)]]
    .reset_index()
)

In [ ]:
submission.head()

In [ ]:
current_time = datetime.now().strftime('%y%m%d%H%M%S')
submission.to_csv("data/submission.csv", index=False)
submission.to_csv(f"data/submission_{current_time}", index=False)